In [2]:
from PIL import Image
import os
from pylab import *
import re
from PIL import Image, ImageChops, ImageEnhance

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [4]:
import csv

In [5]:
path="C:\\Users\\Aman Dhamija\\fake_image_major\\casia-dataset\\CASIA2\\Tp\\Tp_S_NRN_S_O_cha10187_cha10187_12308.jpg"
quality=90

In [8]:
def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename ='resaved/'+filename.split('\\')[-1].split('.')[0] + '.resaved.jpg'
    
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [7]:
x=[]
y=[]

with open('dataset.csv', mode='r') as dataset_file:
    file_reader=csv.reader(dataset_file)
    for row in file_reader:
        x.append(array(convert_to_ela_image(row[0], quality).resize((128, 128))).flatten() / 255.0)
        y.append(row[1])
    

C:\Anaconda3\envs\condaVenv\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag)
C:\Anaconda3\envs\condaVenv\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag)


In [7]:
x = np.array(x)
y = to_categorical(y, 2)

NameError: name 'x' is not defined

In [9]:
x = x.reshape(-1, 128, 128, 3)

In [10]:
x.shape

(9418, 128, 128, 3)

In [11]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state=5)

In [12]:
np.savez_compressed('x_train.npz',x_train)

In [13]:
np.savez_compressed('x_val.npz',x_val)
np.savez_compressed('y_train.npz',y_train)
np.savez_compressed('y_val.npz',y_val)

In [9]:
x1=np.load('x_train.npz')
x2=np.load('x_val.npz')
y1=np.load('y_train.npz')
y2=np.load('y_val.npz')

In [10]:
x_train=x1['arr_0']
x_val=x2['arr_0']
y_train=y1['arr_0']
y_val=y2['arr_0']

In [11]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'valid', 
                 activation ='relu', input_shape = (128,128,3)))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'valid', 
                 activation ='relu'))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.25))
print("Input: ", model.input_shape)
print("Output: ", model.output_shape)

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))




Input:  (None, 128, 128, 3)
Output:  (None, 124, 124, 32)
Input:  (None, 128, 128, 3)
Output:  (None, 120, 120, 32)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Input:  (None, 128, 128, 3)
Output:  (None, 60, 60, 32)


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 32)      25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 60, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 115200)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               29491456  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
__________

In [13]:
optimizer = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)

In [14]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [15]:
early_stopping = EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [16]:
epochs = 30
batch_size = 100

In [17]:
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, 
          validation_data = (x_val, y_val), verbose = 2, callbacks=[early_stopping])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 7534 samples, validate on 1884 samples
Epoch 1/30





 - 1341s - loss: 0.4224 - acc: 0.7843 - val_loss: 0.3355 - val_acc: 0.8806
Epoch 2/30
 - 1294s - loss: 0.3068 - acc: 0.8630 - val_loss: 0.2525 - val_acc: 0.8790
Epoch 3/30
 - 1349s - loss: 0.2441 - acc: 0.8896 - val_loss: 0.2446 - val_acc: 0.8896
Epoch 4/30
 - 1583s - loss: 0.1931 - acc: 0.9139 - val_loss: 0.1660 - val_acc: 0.9315
Epoch 5/30
 - 1534s - loss: 0.1748 - acc: 0.9241 - val_loss: 0.1589 - val_acc: 0.9294
Epoch 6/30
 - 1249s - loss: 0.1586 - acc: 0.9367 - val_loss: 0.1594 - val_acc: 0.9331
Epoch 7/30
 - 1465s - loss: 0.1412 - acc: 0.9407 - val_loss: 0.1789 - val_acc: 0.9214
Epoch 8/30
 - 1404s - loss: 0.1343 - acc: 0.9421 - val_loss: 0.1503 - val_acc: 0.9406
Epoch 9/30
 - 1401s - loss: 0.1127 - acc: 0.9542 - val_loss: 0.1727 - val_acc: 0.9384
Epoch 10/30
 - 1516s - loss: 0.1061 - acc: 0.9574 - val_loss: 0.1845 - val_a

In [18]:
model_json = model.to_json()

In [19]:
with open("model.json","w") as json_file:
    json_file.write(model_json)

In [20]:
model.save_weights("model.h5")